# 🧠 Atlas — Interactive Demo

Welcome! This notebook walks you through the Atlas long-form memory system step by step.

**What you'll learn:**
1. How the agent extracts memories from conversation
2. How memories are stored and retrieved
3. How to test memory recall
4. How to use different providers (Ollama, Gemini, Groq)

> 📊 For full benchmarks and evaluation results, see **[results.ipynb](results.ipynb)**

---

## 1. Setup

First, install dependencies and load the environment.

In [19]:
import os
import sys
import json
from pathlib import Path

# Add project root to path
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.agent import LongMemAgent
from dotenv import load_dotenv

load_dotenv()

print('✓ Dependencies loaded')
print(f'  Project root: {project_root}')

✓ Dependencies loaded
  Project root: /home/pik/dev/atlas


## 2. Choose Your Provider

Atlas supports multiple LLM backends. Pick one:

| Provider | Model | API Key Needed | Notes |
|----------|-------|:--------------:|-------|
| `ollama` | `mistral` | No | Local, fast, free. Run `ollama serve` first |
| `gemini` | `gemma-3-27b-it` | `GEMINI_API_KEY` | Free tier, 30 req/min |
| `groq` | `llama-3.1-8b-instant` | `GROQ_API_KEY` | Very fast, rate limited |

Edit the cell below to change providers.

In [20]:
# ═══════════════════════════════════════════
# 🔧 CONFIGURE YOUR PROVIDER HERE
# ═══════════════════════════════════════════

PROVIDER = 'groq'                 # 'ollama', 'gemini', or 'groq'
# MODEL    = 'mistral'                # model name for your provider
# MODEL  = 'gemma-3-27b-it'         # uncomment for Gemini
MODEL  = 'moonshotai/kimi-k2-instruct-0905'   # uncomment for Groq

# ═══════════════════════════════════════════

# Clean up any previous demo database
demo_db = 'demo_interactive.db'
if os.path.exists(demo_db):
    os.remove(demo_db)
    print(f'🗑️  Cleaned previous {demo_db}')

# Initialize agent
agent = LongMemAgent(
    provider=PROVIDER,
    model=MODEL,
    db_path=demo_db,
    context_limit=4096,
    flush_threshold=0.70
)

print(f'\n✓ Agent initialized')
print(f'  Provider: {agent.provider}')
print(f'  Model:    {agent.model}')
print(f'  DB:       {demo_db}')

🗑️  Cleaned previous demo_interactive.db

✓ Agent initialized
  Provider: groq
  Model:    moonshotai/kimi-k2-instruct-0905
  DB:       demo_interactive.db


## 3. Chat Helper

A nice helper to display conversations with memory metadata.

In [ ]:
from IPython.display import display, Markdown, HTML

def chat(message: str, show_memories=True):
    """Send a message and display the response with memory info."""
    print(f'\n You: {message}')
    
    response = agent.chat(message)
    
    print(f'Atlas: {response["response"]}')
    print(f'\n   ┌─ Turn {response["turn_id"]} │ '
          f'Context: {response["context_utilization"]} │ '
          f'Memories: {response["total_memories"]} │ '
          f'Flush: {"yes" if response["flush_triggered"] else "no"}')
    
    if show_memories and response['active_memories']:
        print(f'   │ 🧠 Retrieved {len(response["active_memories"])} memories:')
        for mem in response['active_memories']:
            print(f'   │   • {mem["content"]} '
                  f'(from turn {mem["origin_turn"]}, conf: {mem["confidence"]:.2f})')
    print('────────────────────────────────────────────────')
    
    
    return response

print('✓ Chat helper ready. Use chat("your message") below!')

✓ Chat helper ready. Use chat("your message") below!


## 4. Plant Some Memories

Let's tell the agent some facts about ourselves. These will be extracted as memories.

In [22]:
chat('Hi! My name is Jordan and I work as a data scientist in Seattle.')
chat("I'm allergic to peanuts and I follow a vegetarian diet.")
chat('My favorite programming language is Python. I also use R for statistics.')
chat('I have a daughter named Maya, she is 5 years old.')


👤 You: Hi! My name is Jordan and I work as a data scientist in Seattle.
🤖 Atlas: Nice to meet you, Jordan—Seattle’s a great hub for data work. How’s the scene treating you these days?

   ┌─ Turn 1 │ Context: 3% │ Memories: 0 │ Flush: no
   └──────────────────────────

👤 You: I'm allergic to peanuts and I follow a vegetarian diet.
🤖 Atlas: Got it—peanuts off the table and vegetarian plates only. Anything in particular you’d like help with right now?

   ┌─ Turn 2 │ Context: 5% │ Memories: 0 │ Flush: no
   └──────────────────────────

👤 You: My favorite programming language is Python. I also use R for statistics.
🤖 Atlas: Solid combo—Python for the heavy lifting and R when you need the statistical muscle.

   ┌─ Turn 3 │ Context: 5% │ Memories: 0 │ Flush: no
   └──────────────────────────

👤 You: I have a daughter named Maya, she is 5 years old.
🤖 Atlas: Five is such a fun age—lots of curiosity and energy. How’s Maya doing?

   ┌─ Turn 4 │ Context: 6% │ Memories: 0 │ Flush: no
   └────

{'response': 'Five is such a fun age—lots of curiosity and energy. How’s Maya doing?',
 'turn_id': 4,
 'context_utilization': '6%',
 'context_tokens': 264,
 'retrieval_ms': 0.0,
 'total_ms': 531.0,
 'flush_triggered': False,
 'total_flushes': 0,
 'active_memories': [],
 'total_memories': 0}

## 5. Trigger Memory Distillation

The agent normally distills memories when context hits 70% capacity.
We can also trigger it manually to extract memories right now.

In [23]:
print('⚡ Triggering memory distillation...\n')

result = agent.manual_distill()

print(f'✓ {result["message"]}')
print(f'  Memories added: {result["memories_added"]}')
print(f'  Total memories: {result["total_memories"]}')

⚡ Triggering memory distillation...

[distiller] Skipping malformed memory item: DistilledMemory.__init__() got an unexpected keyword argument 'reasoning'
[distiller] Skipping malformed memory item: DistilledMemory.__init__() got an unexpected keyword argument 'reasoning'
[distiller] Skipping malformed memory item: DistilledMemory.__init__() got an unexpected keyword argument 'reasoning'
[distiller] Skipping malformed memory item: DistilledMemory.__init__() got an unexpected keyword argument 'reasoning'
[distiller] Skipping malformed memory item: DistilledMemory.__init__() got an unexpected keyword argument 'reasoning'
[distiller] Skipping malformed memory item: DistilledMemory.__init__() got an unexpected keyword argument 'reasoning'
[distiller] Skipping malformed memory item: DistilledMemory.__init__() got an unexpected keyword argument 'reasoning'
[distiller] Skipping malformed memory item: DistilledMemory.__init__() got an unexpected keyword argument 'reasoning'
✓ Distillation comp

## 6. Inspect Stored Memories

Let's see what the distiller extracted.

In [24]:
memories = agent.get_all_memories()

if memories:
    print(f'📦 Stored Memories ({len(memories)} total):\n')
    for m in memories:
        print(f"  [{m['type']:>12}] {m['key']}: {m['value']}")
        print(f"               confidence: {m['confidence']:.2f}, from turn {m['source_turn']}")
else:
    print('No memories stored yet. Run distillation first.')

No memories stored yet. Run distillation first.


## 7. Test Memory Recall

Now the key test — does the agent remember what we told it?

In [25]:
chat("What's my name?")


👤 You: What's my name?
🤖 Atlas: You haven’t told me your name yet—what would you like me to call you?

   ┌─ Turn 5 │ Context: 5% │ Memories: 0 │ Flush: no
   └──────────────────────────


{'response': 'You haven’t told me your name yet—what would you like me to call you?',
 'turn_id': 5,
 'context_utilization': '5%',
 'context_tokens': 202,
 'retrieval_ms': 0.1,
 'total_ms': 722.0,
 'flush_triggered': False,
 'total_flushes': 1,
 'active_memories': [],
 'total_memories': 0}

In [26]:
chat('Suggest a dinner recipe for me.')


👤 You: Suggest a dinner recipe for me.
🤖 Atlas: Quick one-pan lemon-garlic chicken thighs with roasted veggies: toss bone-in thighs, baby potatoes, and broccoli with olive oil, lemon zest, garlic, salt, and pepper; 25 min at 425 °F. While it roasts you can help Maya set the table—she’ll like the bright lemon smell.

   ┌─ Turn 6 │ Context: 7% │ Memories: 0 │ Flush: no
   └──────────────────────────


{'response': 'Quick one-pan lemon-garlic chicken thighs with roasted veggies: toss bone-in thighs, baby potatoes, and broccoli with olive oil, lemon zest, garlic, salt, and pepper; 25 min at 425 °F. While it roasts you can help Maya set the table—she’ll like the bright lemon smell.',
 'turn_id': 6,
 'context_utilization': '7%',
 'context_tokens': 285,
 'retrieval_ms': 0.3,
 'total_ms': 800.5,
 'flush_triggered': False,
 'total_flushes': 1,
 'active_memories': [],
 'total_memories': 0}

In [27]:
chat('What should I get my kid for her birthday?')


👤 You: What should I get my kid for her birthday?
🤖 Atlas: Five-year-olds love open-ended play that lets them build or pretend. A few ideas that usually land:

- Magnetic tile set – bright colors, easy for small hands, grows with her
- Realistic animal figurines (Schleich or similar) – sparks storytelling and pairs with blocks she already has
- Kid-size gardening kit – trowel, gloves, and a couple of seed packets; you can plant together on the weekend
- Story-game board book like “Press Here” or “The Book with No Pictures” – interactive read-alouds she can “control”
- Dress-up trunk add-on: a reversible cape, a few costume masks, and a pair of “magic” gloves—instant superhero/royalty switch-ups

Pick whichever matches her current obsession (animals, building, or make-believe) and you’ll nail it.

   ┌─ Turn 7 │ Context: 12% │ Memories: 0 │ Flush: no
   └──────────────────────────


{'response': 'Five-year-olds love open-ended play that lets them build or pretend. A few ideas that usually land:\n\n- Magnetic tile set – bright colors, easy for small hands, grows with her\n- Realistic animal figurines (Schleich or similar) – sparks storytelling and pairs with blocks she already has\n- Kid-size gardening kit – trowel, gloves, and a couple of seed packets; you can plant together on the weekend\n- Story-game board book like “Press Here” or “The Book with No Pictures” – interactive read-alouds she can “control”\n- Dress-up trunk add-on: a reversible cape, a few costume masks, and a pair of “magic” gloves—instant superhero/royalty switch-ups\n\nPick whichever matches her current obsession (animals, building, or make-believe) and you’ll nail it.',
 'turn_id': 7,
 'context_utilization': '12%',
 'context_tokens': 475,
 'retrieval_ms': 0.4,
 'total_ms': 1487.3,
 'flush_triggered': False,
 'total_flushes': 1,
 'active_memories': [],
 'total_memories': 0}

In [28]:
chat('Tell me everything you remember about me.')


👤 You: Tell me everything you remember about me.
🤖 Atlas: You code in Python and reach for R when you need statistical tools. You have a five-year-old daughter named Maya, and you like quick, practical dinner solutions that leave time to hang out with her.

   ┌─ Turn 8 │ Context: 13% │ Memories: 0 │ Flush: no
   └──────────────────────────


{'response': 'You code in Python and reach for R when you need statistical tools. You have a five-year-old daughter named Maya, and you like quick, practical dinner solutions that leave time to hang out with her.',
 'turn_id': 8,
 'context_utilization': '13%',
 'context_tokens': 532,
 'retrieval_ms': 0.4,
 'total_ms': 828.2,
 'flush_triggered': False,
 'total_flushes': 1,
 'active_memories': [],
 'total_memories': 0}

## 8. Try It Yourself!

Use the cell below to have your own conversation with the agent.
Each message is processed through the full memory pipeline.

In [29]:
# Type your message here:
chat('What do I do for work?')


👤 You: What do I do for work?
🤖 Atlas: You haven’t mentioned your job yet—what field are you in?

   ┌─ Turn 9 │ Context: 14% │ Memories: 0 │ Flush: no
   └──────────────────────────


{'response': 'You haven’t mentioned your job yet—what field are you in?',
 'turn_id': 9,
 'context_utilization': '14%',
 'context_tokens': 561,
 'retrieval_ms': 0.4,
 'total_ms': 756.2,
 'flush_triggered': False,
 'total_flushes': 1,
 'active_memories': [],
 'total_memories': 0}

In [30]:
# Try another:
chat('Your message here')


👤 You: Your message here
🤖 Atlas: Got it—thanks for the update.

   ┌─ Turn 10 │ Context: 14% │ Memories: 0 │ Flush: no
   └──────────────────────────


{'response': 'Got it—thanks for the update.',
 'turn_id': 10,
 'context_utilization': '14%',
 'context_tokens': 580,
 'retrieval_ms': 0.5,
 'total_ms': 541.3,
 'flush_triggered': False,
 'total_flushes': 1,
 'active_memories': [],
 'total_memories': 0}

## 9. Database Peek

Under the hood, memories are stored in SQLite with vector embeddings + full-text search indexes.

In [31]:
import sqlite3

conn = sqlite3.connect(demo_db)
cursor = conn.cursor()

cursor.execute('SELECT COUNT(*) FROM memories WHERE is_active = 1')
active = cursor.fetchone()[0]

cursor.execute('SELECT COUNT(*) FROM turns')
turns = cursor.fetchone()[0]

cursor.execute('''
    SELECT type, COUNT(*) as count 
    FROM memories WHERE is_active = 1 
    GROUP BY type
''')
types = cursor.fetchall()

print(f'📋 Database: {demo_db}')
print(f'   Active memories: {active}')
print(f'   Total turns: {turns}')
print(f'   Memory types:')
for type_name, count in types:
    print(f'     {type_name}: {count}')

conn.close()

📋 Database: demo_interactive.db
   Active memories: 0
   Total turns: 10
   Memory types:


---

## Next Steps

| What | How |
|:-----|:----|
| 📊 **See benchmarks** | Open [results.ipynb](results.ipynb) |
| 💬 **Interactive CLI** | `uv run python main.py` |
| 🧪 **Run evaluation** | `uv run python eval/evaluate.py --provider gemini` |
| 🏠 **Local eval** | `uv run python eval/evaluate.py --local --model mistral` |
| 📖 **Quick eval** | `uv run python eval/evaluate.py --quick --local` |

### Architecture

```
User Message
    │
    ▼
┌─────────────────────────┐
│    Context Manager      │ ← checks if context needs flushing
│    (token counting)     │
└────────┬────────────────┘
         │ if >70% full
         ▼
┌─────────────────────────┐
│    Memory Distiller     │ ← LLM extracts memories from conversation
│    (Groq/Gemini/Ollama) │
└────────┬────────────────┘
         │
         ▼
┌─────────────────────────┐
│    Memory Store         │ ← SQLite + sqlite-vec + FTS5
│    (persist & search)   │
└────────┬────────────────┘
         │
         ▼
┌─────────────────────────┐
│    Memory Retriever     │ ← hybrid vector + keyword search
│    (RRF fusion)         │
└────────┬────────────────┘
         │
         ▼
┌─────────────────────────┐
│    System Prompt        │ ← memories injected into context
│    + LLM Response       │
└─────────────────────────┘
```

## Cleanup (Optional)

In [32]:
# Uncomment to delete the demo database:
# os.remove(demo_db)
# print(f'✓ Removed {demo_db}')

## NOTE: To check the actual metrics, check out results.ipynb